In [1]:
import numpy as np
import math

# Linear Regression

### Some Methods

In [2]:
# let's understand reshape() first
# the value of -1 is a freecard that numpy calculates automatically

y = np.array([1, 2, 3, 4, 5, 6]) # 6 elements in the original vector
print("When the other dimension is 3, the shape of -1 will be", y.reshape(-1, 3).shape[0]) # here -1 will be (6/3) = 2
print("When the other dimension is 2, the shape of -1 will be", y.reshape(2, -1).shape[1]) # here -1 will be (6/2) = 3
print("When the other dimension is 6, the shape of -1 will be", y.reshape(-1, 6).shape[0]) # here -1 will be (6/6) = 1

When the other dimension is 3, the shape of -1 will be 2
When the other dimension is 2, the shape of -1 will be 3
When the other dimension is 6, the shape of -1 will be 1


In [7]:
# now we want to reshape y, which is a 1D vector, into an 2D array with 1 column
y = np.array([1, 2, 3])
print("-------------------------------------")
print("The shape of y before reshaping:", y.shape)
print("y before reshaping:")
print(y)
print("-------------------------------------")
y_reshaped = y.reshape(-1, 1)
print("The shape of y after reshaping:", y_reshaped.shape)
print("y after reshaping:")
print(y_reshaped)
print("-------------------------------------")

-------------------------------------
The shape of y before reshaping: (3,)
y before reshapeing:
[1 2 3]
-------------------------------------
The shape of y after reshaping: (3, 1)
y after reshapeing:
[[1]
 [2]
 [3]]
-------------------------------------


In [11]:
# now, the flatten() method converts multidimentional array into a vector
a = np.array([[1, 2, 3],
     [2, 3, 4],
     [4, 5, 6]])
print(a.flatten().tolist())

[1, 2, 3, 2, 3, 4, 4, 5, 6]


### Closed Form Solution

In [12]:
# close form
# (XTX)-1XTY
def linear_regression_close(X, y):
    X = np.array(X)
    y = np.array(y).reshape(-1, 1)
    theta = np.linalg.inv(X.T @ X) @ X.T @ y
    return np.round(theta, 4).flatten().tolist()

X = [[1, 1], [1, 2], [1, 3]]
y = [1, 2, 3]
print(linear_regression_close(X, y))

[-0.0, 1.0]


### Gradient Descent Solution

**1. Cost Function:**

$$J(\theta) = \frac{1}{2m} \sum_{i=1}^{m} (\hat{y}_{i} - y_{i})^2 = \frac{1}{2m} \sum_{i=1}^{m} (\text{error}_{i})^2$$

**2. Taking the Derivative (Chain Rule):**

$$\frac{\partial J}{\partial \theta} = \frac{1}{2m} \sum_{i=1}^{m} 2 \cdot \text{error}_{i} \cdot \frac{\partial}{\partial \theta}(\text{error}_{i})$$

**3. Derivative of Error:**

$$\frac{\partial}{\partial \theta}(\text{error}) = \frac{\partial}{\partial \theta}(X\theta) = X^T$$

**Therefore:**

$$\frac{\partial J}{\partial \theta} = \frac{1}{m} X^T \cdot \text{error}$$

In [12]:
# gradient descent
def linear_regression_gradient(X, y, alpha=0.01, iterations=1000):
    X = np.array(X) 
    y = np.array(y).reshape(-1, 1)
    
    m, n = X.shape
    theta = np.zeros((n, 1)) 
    for _ in range(iterations):
        predictions = X @ theta
        errors = predictions - y
        gradient = X.T @ errors / m
        theta -= alpha * gradient
    return np.round(theta, 4).flatten().tolist()

X = [[1, 1], [1, 2], [1, 3]]
y = [1, 2, 3]
print(linear_regression_gradient(X, y))

[0.1107, 0.9513]


### PyTorch Version

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

# torch.randn generates random points with mean=0, variance=1
# (100, 1) is the shape of the generated tensor
X = torch.randn(100, 1) * 10

# generates y using linear relationship with noise
y = 3 * X + 2 + torch.randn(100, 1)  

# create a new class that inherits from nn.Module
class LinearRegression(nn.Module):  
    def __init__(self):
        # super().__init__() calls the constructor of the parent class (nn.Module)
        super().__init__()
        
        # nn.Linear(n, m) creates a linear layer
        # the input will be in shape (batch_size, n)
        # the output will be in shape (batch_size, m)
        # the calculation will be output = input @ weight.T + bias
        # therefore, the weight of the layer will be in shape (m, n)
        # accordingly, the bias will be in shape (m)
        self.linear = nn.Linear(1, 1)
    
    # forward(self, x) function decides what goes through in this layer
    def forward(self, x):
        return self.linear(x)

# create model, loss function, and optimizer
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# training loop
for epoch in range(2000):
    # 1. forward pass: compute predictions
    # model(X) calls the __call__() method defined in the parent class (nn.Module)
    # in __call__(), PyTorch calls the forward() method
    # PyTorch also registers gradient operation in __call__()
    predictions = model(X)
    
    # 2. compute loss
    loss = criterion(predictions, y)
    
    # 3. zero gradients
    # this is necessary because the backward() method accumulates gradients as default
    # if we do not zero grad, backward() adds the current gradient to previous gradients
    optimizer.zero_grad()
    
    # 4. backward pass: compute gradients
    loss.backward()
    
    # 5. update weights
    optimizer.step()
    
    if (epoch + 1) % 200 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

# print learned parameters
# here we use item() to get values from PyTorch tensors
# Note: this is only possible when the tensor has a single value
# when a tensor has multiple values, convert to list by weight.data.tolist() 
# alternatively, convert to numpy array by weight.data.numpy()
print(f'\nWeight: {model.linear.weight.item():.4f}')
print(f'Bias: {model.linear.bias.item():.4f}')

Epoch 200, Loss: 1.6337
Epoch 400, Loss: 1.2722
Epoch 600, Loss: 1.1096
Epoch 800, Loss: 1.0365
Epoch 1000, Loss: 1.0036
Epoch 1200, Loss: 0.9888
Epoch 1400, Loss: 0.9821
Epoch 1600, Loss: 0.9791
Epoch 1800, Loss: 0.9778
Epoch 2000, Loss: 0.9772

Weight: 2.9821
Bias: 1.8742


# logistic regression

In [9]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
    
def logistic_regression(X, y, alpha=0.01, iterations=1000):
    X = np.array(X)
    y = np.array(y).reshape(-1, 1)
    m, n = X.shape
    theta = np.zeros((n, 1))
    
    for _ in range(iterations):
        z = X @ theta
        predictions = sigmoid(z)
        errors = predictions - y
        gradients = X.T @ errors / m
        theta -= alpha * gradients
    return np.round(theta, 4).flatten().tolist()

X = [[1, 1], [1, 2], [1, 3]]
y = [1, 0, 1]
print(logistic_regression(X, y))

[0.2529, 0.1954]


# gradient descent

In [64]:
def loss_fn(n, x):
    return (n - x ** 2) ** 2

def gradient(n, x):
    return -4 * x * (n - x ** 2)

def gradient_descent(n, lr=0.01):
    x = n 
    
    if n > 1000: lr *= 10
    
    for i in range(1000):
        loss = loss_fn(n, x)
        if abs(loss) < 1e-10: 
            break
        grad = gradient(n, x)
        grad = max(min(grad, 100), -100)
        x -= grad * lr
        
    return x

gradient_descent(10000)

100.0

# K-means

In [72]:
def distance(a, b):
    return np.sum((a - b) ** 2) ** 0.5

def k_means(points: list[tuple[float, float]],
            k: int, 
            initial_centroids: list[tuple[float, float]], 
            max_iterations: int) -> list[tuple[float, float]]:
    points = np.array(points)
    centroids = np.array(initial_centroids)
    
    for _ in range(max_iterations):
        assignments = []
        for point in points:
            distances = np.array([distance(point, centroid) for centroid in centroids])
            assignments.append(np.argmin(distances))
        assignments = np.array(assignments)

        new_centroids = []
        for cluster in range(k):
            subset = points[assignments == cluster]
            if len(subset) > 0:
                new_centroids.append(subset.mean(axis=0))
            else:
                new_centroids.append(centroids[cluster])
        new_centroids = np.array(new_centroids)

        if np.all(new_centroids == centroids):
            break
        else:
            centroids = new_centroids
    
    return np.round(centroids, 4).tolist()
    
    
points = [(1, 2), (1, 4), (1, 0), (10, 2), (10, 4), (10, 0)]
k = 2
initial_centroids = [(1, 1), (10, 1)]
max_iterations = 10
k_means(points, k, initial_centroids, max_iterations)

[[1.0, 2.0], [10.0, 2.0]]

# Attention

n = number of heads

B = batch size

L = sequence length

D = hidden dimension

d = head dimension = D / n

X: [B, L, D]

Q, K, V: [B, n, L, d]

In [90]:
# single head attention
def compute_qkv(X, W_q, W_k, W_v):
    Q = X @ W_q
    K = X @ W_k
    V = X @ W_v
    return Q, K, V

def self_attention(Q, K, V):
    d_k = Q.shape[1]
    scores = Q @ K.T / np.sqrt(d_k)
    score_max = np.max(scores, axis=1, keepdims=True)
    weights = np.exp(scores - score_max) / np.sum(np.exp(scores - score_max), axis=1, keepdims=True)
    attention = weights @ V
    return attention

X = np.array([[1, 0], [0, 1]])
W_q = np.array([[1, 0], [0, 1]])
W_k = np.array([[1, 0], [0, 1]])
W_v = np.array([[1, 2], [3, 4]])
Q, K, V = compute_qkv(X, W_q, W_k, W_v)
self_attention(Q, K, V)

array([[1.6604769, 2.6604769],
       [2.3395231, 3.3395231]])

In [95]:
def multi_head_attention(Q, K, V, n_heads):
    d_model = Q.shape[1]
    assert d_model % n_heads == 0
    d_k = d_model // n_heads
    
    Q_reshape = Q.reshape(Q.shape[0], n_heads, d_k).transpose(1, 0, 2)
    K_reshape = K.reshape(K.shape[0], n_heads, d_k).transpose(1, 0, 2)
    V_reshape = V.reshape(V.shape[0], n_heads, d_k).transpose(1, 0, 2)
    
    attentions = []
    for i in range(n_heads):
        attn = self_attention(Q_reshape[i], K_reshape[i], V_reshape[i])
        attentions.append(attn)
    
    attention_output = np.concatenate(attentions, axis=1)
    return attention_output

In [39]:
m, n = 4, 4 
n_heads = 2 
np.random.seed(42) 
X = np.arange(m*n).reshape(m,n) 
X = np.random.permutation(X.flatten()).reshape(m, n) 
W_q = np.random.randint(0,4,size=(n,n)) 
W_k = np.random.randint(0,5,size=(n,n)) 
W_v = np.random.randint(0,6,size=(n,n)) 
Q, K, V = compute_qkv(X, W_q, W_k, W_v) 
print(multi_head_attention(Q, K, V, n_heads))

[[103. 109.  46.  99.]
 [103. 109.  46.  99.]
 [103. 109.  46.  99.]
 [103. 109.  46.  99.]]


## Decision Tree

In [97]:
import math
from collections import Counter

In [102]:
def calculate_entropy(labels):
    label_counts = Counter(labels)
    total_count = len(labels)
    entropy = -sum((count / total_count) * math.log2(count / total_count) for count in label_counts.values())
    return entropy

def calculate_info_gain(examples, attr, target_attr):
    total_entropy = calculate_entropy([example[target_attr] for example in examples])
    values = set(example[attr] for example in examples)
    attr_entropy = 0
    for value in values:
        value_subset = [example[target_attr] for example in examples if example[attr] == value]
        value_entropy = calculate_entropy(value_subset)
        attr_entropy += (len(value_subset) / len(examples)) * value_entropy
    return total_entropy - attr_entropy

def majority_class(examples, target_attr):
    labels = [example[target_attr] for example in examples]
    return max(labels, key=lambda x: labels.count(x))

def learn_decision_tree(examples, attributes, target_attr):
    if not examples: return "no examples"
    if all(example[target_attr] == examples[0][target_attr] for example in examples):
        return examples[0][target_attr]
    if not attributes:
        return majority_class(examples, target_attr)
    
    gains = {attr: calculate_info_gain(examples, attr, target_attr) for attr in attributes}
    best_attr = max(gains, key=gains.get)
    tree = {best_attr: {}}
    
    for value in set(example[best_attr] for example in examples):
        subset = [example for example in examples if example[best_attr] == value]
        new_attributes = attributes.copy()
        new_attributes.remove(best_attr)
        subtree = learn_decision_tree(subset, new_attributes, target_attr)
        tree[best_attr][value] = subtree
        
    return tree

In [103]:
print(learn_decision_tree([ {'Outlook': 'Sunny', 'Wind': 'Weak', 'PlayTennis': 'No'}, {'Outlook': 'Overcast', 'Wind': 'Strong', 'PlayTennis': 'Yes'}, {'Outlook': 'Rain', 'Wind': 'Weak', 'PlayTennis': 'Yes'}, {'Outlook': 'Sunny', 'Wind': 'Strong', 'PlayTennis': 'No'}, {'Outlook': 'Sunny', 'Wind': 'Weak', 'PlayTennis': 'Yes'}, {'Outlook': 'Overcast', 'Wind': 'Weak', 'PlayTennis': 'Yes'}, {'Outlook': 'Rain', 'Wind': 'Strong', 'PlayTennis': 'No'}, {'Outlook': 'Rain', 'Wind': 'Weak', 'PlayTennis': 'Yes'} ], ['Outlook', 'Wind'], 'PlayTennis'))

{'Outlook': {'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}, 'Sunny': {'Wind': {'Weak': 'No', 'Strong': 'No'}}}}


## PCA

In [2]:
def pca(data, k):
    n_data = (data - np.mean(data, axis=0)) / np.std(data, axis=0)
    covariance_matrix = np.cov(n_data, rowvar=False)
    
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    
    idx = np.argsort(-eigenvalues)
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    principal_components = eigenvectors[:, :k]
    return np.round(principal_components, 4)

print(pca(np.array([[4,2,1],[5,6,7],[9,12,1],[4,6,7]]),2))

[[-0.0188  0.7675]
 [ 0.3687  0.0898]
 [-0.8084 -0.2955]
 [ 0.4585 -0.5618]]


## torch

In [72]:
import torch
import torch.nn as nn

class TwoLayerFC(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
   
    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)

## layer norm

In [ ]:
def layer_norm(x, beta, gamma):
    x_mean = np.mean(x, axis=1, keepdims=True)
    x_std = np.std(x, axis=1, keepdims=True)
    normed_x = (x - x_mean) / (x_std + 1e-8)
    normed_x = gamma * normed_x + beta
    return normed_x

In [ ]:
import torch
from torch import nn

class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.o_linear = nn.Linear(d_model, d_model)
        
    def split_heads(self, x, batch_size):
        x = x.view(batch_size, -1, self.num_heads, self.head_dim)
        return x.permute(0, 2, 1, 3)
    
    def forward(self, hidden_state, attention_mask=None):
        batch_size = hidden_state.size()[0]
        query = self.q_linear(hidden_state)
        key = self.k_linear(hidden_state)
        value = self.v_linear(hidden_state)
        query = split_heads(query, batch_size)
        key = split_heads(key, batch_size)
        value = split_heads(value, batch_size)
        attention_scores = torch.matmul(query, key.transpose(-1, -2)) / torch.sqrt(torch.tensor(self.head_dim))
        
        if attention_mask != None:
            attention_scores += attention_mask * -1e9
            
        attention_probs = torch.softmax(attention_scores, dim=-1)
        output = torch.matmul(attention_probs, value)
        
        output = output.transpose(-1, -2).contiguous().view(batch_size, -1, self.head_dim * self.num_heads)
        output = self.o_linear(output)
        return output

In [ ]:
import torch
import torch.nn as nn

class TwoLayerFC(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        x = self.relu(self.fc1(x))
        return self.fc2(x)